# 1. 키움 OpenAPI

## [통신동작]
 - 키움OpenAPI는 모두 비동식으로 동작
 - 수신데이터를 가져오려면 반드시 이벤트 함수안에 데이터 획득 함수 구현
 - 요청함수 ---> 이벤트 함수 ---> 데이터 획득 함수 --> 이벤트 함수 리턴

## [화면번호]
 - 화면번호는 서버에 시세조회나 주문을 요청할때 이 요청을 구별하기 위한 키값
 - 0000(혹은 0)을 제외한 임의의 숫자를 사용하시면 되는데 갯수가 200개로 한정
 - 사용하는 화면번호가 200개를 넘는 경우 원하지 않는 결과를 나타날 수 있음

## [이벤트 함수]
 - 서버에 데이터 요청 후 키움OpenAPI가 데이터를 수신하게 되면 사용자 프로그램에 특정함수를 자동으로 호출
 - OnReceiveTRData(), OnEventConnect(), OnReceiveRealData() 처럼 앞에 On~이 붙어있음
 - 로그인, 시세조회, 주문등 각 요청별로 요청함수, 이벤트와 데이터획득함수가 분류되어있음

# 2. 로그인 버전처리
## [로그인 개요]
- 단순히 CommConnect()함수를 호출하면 됨
- 로그인과 관련한 이벤트 함수는 OnEventConnect()
- 전달되는 인자값이 0이면 로그인 성공이며 나머지 경우엔 에러코드와 에러메시지 내용으로 원인을 파악 가능
- 로그인은 로그인창에서 운영서버(실제서버)와 모의투자 선택 가능

## [수동 로그인]
- 로그인창에 로그인ID와 비밀번호를 직접입력해서 로그인(Default)
- 수동 로그인중에 버전처리내용이 있는 경우 버전처리도 함께 진행됩니다.

## [자동 로그인]
- 로그인한 다음 계좌비밀번호 입력창을 통해 자동로그인을 설정 가능
- 트레이 메뉴(모니터 오른쪽 하단)에서 "계좌비밀번호 저장" 메뉴를 선택후 계좌 비밀번호 입력
- 자동 로그인중에 버전처리내용이 있는 경우 버전처리도 함께 진행됩니다.(2017.03.30 이후)
<img src='auto_login_pic.png'>

## [버전 처리] ?
- 로그인중에는 버전처리가 함께 처리될 수 있는데 이때"버전처리를 받으시려면 현재 실행 중인 OpenAPI OCX를 탑재한 프로그램을 종료하신후
확인버튼을 눌러주시기 바랍니다.
- 그대로 진행시 버전처리가 정상적으로 실행되지 않습니다."안내 메시지창을 출력됩니다.
- 이 메시지창에 소개된 대로 KOA Studio를 포함해서 실행중인 모든 OpenAPI프로그램을 먼저 종료해주시고 버전처리를 진행하셔야 합니다.
- 그렇지 않으면 버전처리가 안되며 버전처리가 완료될때까지 같은 메시지 창이 출력됩니다.

## [모의투자]
- 로그인 창에서 모의투자접속을 선택을 체크하면 모의투자로 접속하며 체크를 풀면 운영서버로 접속합니다.
- KOA Studio 프로그램은 항상 모의투자로만 접속 가능

## [기타]
- 한 PC에 같은ID로 중복해서 운영서버에 로그인하면 마지막에 로그인한 경우만 유지되고 이전에 로그인한 프로그램은 자동으로 로그오프됩니다.
- 그러나 다른 PC를 이용해서 중복로그인 하거나 같은 PC에 모의투자로 로그인하게 되면 모두 로그인 상태를 유지하게 됩니다.
- (2017.03.30 이후 - 실서버 1, 모의 1 총 2 계정 가능)

# 3. 조회와 실시간 데이터 처리
## [조회처리(조회요청)]
- 키움OpenAPI가 제공하는 데이터중에서 원하는 데이터를 서버에 요청해서 가져오는 것
- TR(Transaction)단위로 처리 : 서버와 데이터를 주고받을때 정의한 약속된 규약이며 입력부분(Input)과 출력부분(Output)으로 구성
- 입력부분은 요청하는 데이터에 따라 입력갯수(입력항목)가 달라지며, 출력부분은 보통 데이터갯수(출력항목)가 여러개로 구성

- 출력부분은 출력항목이 한번씩만 전달되는 싱글데이터와 복수로 전달되는 멀티데이터가 있고 TR에 따라 다름
- TR은 KOA Studio의 TR목록 탭에서 찾아볼 수 있음

## [조회제한]
- 키움OpenAPI에서 시세조회는 1초당 5회로 제한되는데 관심종목 조회와 조건검색 조회 횟수가 합산됨
- 1초 동안 시세조회2회 관심종목 1회 조건검색 2회 순서로 조회를 했다면 모두 합쳐서 5회이므로 모두 조회성공하겠지만 
- 조건검색을 3회 조회하면 맨 마지막 조건검색 조회는 실패

## [연속조회]
- TR별로 한번에 전달할 수 있는 데이터 갯수가 정해져 있으며, 이 갯수보다 데이터가 많을 때 연속조회 사용
- 연속조회하는 방법은 CommRqData()에서 인자값만 바꿔주면 비교적 쉽게 처리 가능
- CommRqData("일별데이터조회", "OPT10086" , 0, "0001"); // 처음조회시 혹은 연속데이터가 없을때
- CommRqData("일별데이터조회", "OPT10086" , 2, "0001"); // 연속조회시

## [실시간 데이터]
- 조회요청이 성공하면 관련 실시간 데이터를 서버에서 자동으로 OnReceiveRealData()이벤트 함수로 전달
- 키움OpenAPI에서는 실시간 데이터가 전달될때 실시간 타입단위로 전달됩니다.
- 실시간 타입 : KOA Studio의 실시간 탭의 실시간 목록에서 Real Type에 주식시세에서 종목프로그램매매까지 나열되어 있는데

# 4. 주문과 잔고처리
## [개요]
- 국내주식과 코스피200 지수선물/옵션, 주식선물을 거래 가능
- 정정주문은 원주문에 대한 수량정정과 가격정정만 가능
- 거래구분을 변경하는 정정주문은 지원하지 않음
- 상품별로 전용 주문함수가 있으며 국내주식 주문의 경우 SendOrderCredit()함수를 이용해서 대주를 제외한 신용주문도 지원


## [주문]
- 주문을 내면 OnReceiveTRData(), OnReceiveMsg(), OnReceiveChejan()이벤트 함수가 차례로 호출
- OnReceiveTRData()이벤트 함수는 주로 조회요청후 호출되는 함수이지만 주문시에도 호출되는데 이 이벤트내부에서 주문번호를
얻어올 수 있습니다.그런데 만일 주문이 실패하게 되면 주문번호는 공백("")으로 전달됩니다.
- 상품별 주문함수는 리턴값이 0(성공)이여도 장 개시전 주문이나 호가범위를 벗어난 주문등 주문이 실패할 경우는 다양합니다.
- 보유하고 있는 현금을 넘는 수량으로 주문한 경우 자동으로 미수로 계산되므로 [0398] 계좌증거금률 변경등록 화면을 통해 
100%현금 주문만 가능하도록 설정할 수도 있습니다.

## [주문제한]
- 국내 주식주문과 국내 주식신용주문, 선물옵션주문은 모두 1초당 5회로 제한

## [주문체결, 잔고]
- 주문과 관련한 이벤트 함수는 OnReceiveMsg(), OnReceiveTRData(), OnReceiveChejan()이렇게 3개
- OnReceiveMsg()이벤트 함수는 주문성공, 실패 메시지를 코드와 함께 전달하므로 상세한 내용을 파악 가능
- OnReceiveTRData()이벤트 함수는 주문후 호출되며 주문번호를 얻을수 있습니다.만약 이 이벤트 함수에서 주문번호를 얻을수 없으면
해당 주문은 실패한 것입니다.
- OnReceiveChejan()이벤트 함수는 주문접수, 체결, 잔고발생시 호출되며 이 이벤트 함수를 통해 대부분의 주문관련
정보를 얻을 수 있습니다.
- 주문요청에 대한 응답은 주문접수, 주문체결, 잔고수신 순서로 진행되며, 주문번호는 처음 접수됬을때 한번 부여되지만 
체결번호는 체결될때 마다 체번되서 전달
- 주문 ---> 접수 ---> 체결1 ---> 잔고1  ---> 체결2  ---> 잔고2... ---> 체결n  ---> 잔고n
- 주문에 대한 자세한 내용은 OnReceiveChejanData()이벤트 함수가 호출될때 전달되는 sGubun값, sFidList값을 이용하는데 
sGubun값은 접수와 체결시 '0'값, 잔고전달은 '1'값을 가지게 됩니다. 이값에 따라 ';'로 연결된 sFidList값도 달라지는데 이 값을
파싱해서 GetChejanData()함수호출시 인자로 사용하시면 보다 상세한 내용을 얻을 수 있습니다.

## [OnReceiveChejan()이벤트 함수로 전달되는 FID목록정리]
- "9203" : "주문번호" 
- "302" : "종목명" 
- "900" : "주문수량" 
- "901" : "주문가격" 
- "902" : "미체결수량" 
- ...

# 5. 조건검색
## [조건검색 개요]
- 키움OpenAPI에서 제공하는 조건검색 기능은 영웅문HTS에서 작성 조건식을 불러서 사용하는 방식
- 조건검색 관련 6개 함수와 3개 이벤트 함수가 제공되며 이를 이용해서 조건검색과 실시간 조건검색(반복적인 조건검색 요청없이 
자동으로 신규종목 편입, 이탈되는 기능)을 설정할수 있습니다.
- 영웅문HTS와 동일하게 실시간 조건검색은 최대 10개 조건식만 실시간 조건검색으로 요청 가능
- 조건검색 결과가 100종목을 넘게 되면 실시간 조건검색을 할수가 없음

## [조건검색 제한]
- 조건검색(실시간 조건검색 포함)은 시세조회와 관심종목조회와 합산해서 1초에 5회만 요청 가능하며 1분에 1회로 조건검색 제한
- **첫번째 제한조건** : 1초에 5회만 조회가능
- **두번째 제한조건** : 조건별 1분당 1회로 제한(실시간 조건검색 수신에는 영향없음)
<pre>
    09:00:00  조회 시작
    1번부터 5번 조건식은 조회성공(첫번째 제한조건, 두번째 제한조건 모두 만족)
    6번부터 10번 조건식은 조회실패(첫번째 제한조건)

    09:00:01 조회 재시작(1초후 재조회)
    1번부터 5번 조건식은 조회실패(두번째 제한조건)
    6번부터 10번 조건식은 조회성공(첫번째 제한조건, 두번째 제한조건 모두 만족)
    ... ... ...

    09:01:00 조회시작 (첫조회 1분후)
    1번부터 5번 조건식은 조회성공(첫번째 제한조건, 두번째 제한조건 모두 만족)
    6번부터 10번 조건식은 조회실패(첫번째 제한조건)

    09:00:01 조회 재시작(1분 1초후 재조회)
    1번부터 5번 조건식은 조회실패(두번째 제한조건)
    6번부터 10번 조건식은 조회성공(첫번째 제한조건, 두번째 제한조건 모두 만족)
</pre>

## [실시간 조건검색]
- 실시간 조건검색 결과로 100종목 이상이 검색되는 조건식은 실시간 조건검색 실행이 안됨
- 실시간 조건검색은 모두 10개 조건식만 사용 가능

# 6. 기타함수

## 종목정보 관련함수

# 7. 키움 OpenAPI 오류코드

# 8. 실전프로그래밍 예제

```Python
class Kiwoom(QAxWidget):
    def __init__(self):
        self.setControl("KHOPENAPI.KHOpenAPICtrl.1")
```
----------------------------

```python
        # 이벤트 함수 등록
        self.OnEventConnect.connect(self.On_EventConnect)
        self.OnReceiveTrData.connect(self.On_ReceiveTrData)
        self.OnReceiveChejanData.connect(self.On_ReceiveChejanData)
        ```

## 접속 방법
 1. CommConnect()
 2. On_EventConnect()

```Python
# 접속 시
def CommConnect(self):
    self.dynamicCall("CommConnect()")

    self.login_event_loop = QEventLoop()
    self.login_event_loop.exec_()

def On_EventConnect(self, errCode):
    if errCode == 0:
        print("connected")
    else:
        print("disconnected")
    self.login_event_loop.exit()
```

## 데이터 조회
 1. SetInputValue()
 2. CommRqData()
 3. On_ReceiveTrData()
     4. CommGetData()

```Python
def SetInputValue(self, sID, sValue):
    self.dynamicCall("SetInputValue(QString, QString)", sID, sValue)

def CommRqData(self, sRQName, sTRCode, nPrevNext, sScreenNo):
    self.dynamicCall("CommRqData(QString, QString, int, QString)", sRQName, sTRCode, nPrevNext, sScreenNo)

    self.tr_event_loop = QEventLoop()
    self.tr_event_loop.exec_()
```
----------------------------

```Python
def On_ReceiveTrData(self, ScrNo, RQName, TrCode, RecordName, PrevNext, DataLength, ErrorCode, Message, SplmMsg):
    self.prev_next = PrevNext

    if RQName == "opt10081_req":
        cnt = self.GetRepeatCnt(TrCode, RQName)

        for i in range(cnt):
            date = self.CommGetData(TrCode, "", RQName, i, "일자")
            open = self.CommGetData(TrCode, "", RQName, i, "시가")
            high = self.CommGetData(TrCode, "", RQName, i, "고가")
            low = self.CommGetData(TrCode, "", RQName, i, "저가")
            close = self.CommGetData(TrCode, "", RQName, i, "현재가")
            volume = self.CommGetData(TrCode, "", RQName, i, "거래량")
```
----------------------------

```Python
def CommGetData(self, sJongmokCode, sRealType, sFieldName, nIndex, sInnerFiledName):
    data = self.dynamicCall("CommGetData(QString, QString, QString, int, QString)", sJongmokCode, sRealType,
                            sFieldName, nIndex, sInnerFiledName)
    return data.strip()
```
----------------------

```Python
def GetRepeatCnt(self, sTrCode, sRecordName):
    ret = self.dynamicCall("GetRepeatCnt(QString, QString)", sTrCode, sRecordName)
    return ret
```

## 주문 및 체결
 1. OnReceiveTRData()
 2. OnReceiveMsg()
 3. OnReceiveChejan()

```Python
#매수
if split_row_data[-1].rstrip() == '매수전':
    self.kiwoom.SendOrder("SendOrder_req", "0101", account, 1, code, num, price, hoga_lookup[hoga], "")
#매도
if split_row_data[-1].rstrip() == '매도전':
    self.kiwoom.SendOrder("SendOrder_req", "0101", account, 2, code, num, price, hoga_lookup[hoga], "")
```

----------------------

```Python
def SendOrder(self, sRQName, sScreenNo, sAccNo, nOrderType, sCode, nQty, nPrice, sHogaGb, sOrgOrderNo):
    self.dynamicCall("SendOrder(QString, QString, QString, int, QString, int, int, QString, QString)",
                     [sRQName, sScreenNo, sAccNo, nOrderType, sCode, nQty, nPrice, sHogaGb, sOrgOrderNo])


def On_ReceiveChejanData(self, sGubun, nItemCnt, sFidList):
    print("sGubun: ", sGubun)
    print(self.GetChejanData(9203))
    print(self.GetChejanData(302))
    print(self.GetChejanData(900))
    print(self.GetChejanData(901))
```

-----------------------------

```Python
def GetChejanData(self, nFid):
        cmd = 'GetChejanData("%s")' % nFid
        ret = self.dynamicCall(cmd)
        return ret
```